In [6]:
label_dict = {0: 'rumor',
 1: 'hate',
 2: 'unreliable',
 3: 'conspiracy',
 4: 'clickbait',
 5: 'satire',
 6: 'fake',
 7: 'reliable',
 8: 'bias',
 9: 'political',
 10: 'junksci',
 11: 'unknown'}

In [7]:
import numpy as np

In [23]:
import torch

In [26]:
!conda install pandas
import pandas as pd

Solving environment: done

# All requested packages already installed.



ModuleNotFoundError: No module named 'pandas'

In [11]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


In [10]:
from transformers import BertTokenizer, BertForSequenceClassification


ModuleNotFoundError: No module named 'transformers'

In [7]:
from torch.utils.data import TensorDataset

In [8]:
data = [["We have created various performance enhancements that have significantly decreased our training time, server startup time, and text analysis times."]]
validate_data = pd.DataFrame(data, columns= ['content'])

NameError: name 'pd' is not defined

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

encoded_data_val = tokenizer.batch_encode_plus(
    validate_data.content.values,  #article text being passed to encoder
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=500, 
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
dataset_val = TensorDataset(input_ids_val, attention_masks_val)

NameError: name 'BertTokenizer' is not defined

In [11]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 2

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

ModuleNotFoundError: No module named 'torch'

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=12,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

#file with model state is being loaded and used here

model.load_state_dict(torch.load('finetuned_BERT_epoch_3.model', map_location=torch.device('cpu')))

model.eval()



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [19]:
predictions = []
for batch in dataloader_validation:

    batch = tuple(b.to(device) for b in batch)

    inputs = {'input_ids':      batch[0],'attention_mask': batch[1],}

    with torch.no_grad():        
        outputs = model(**inputs)

    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    #label_ids = inputs['labels'].cpu().numpy()
    predictions.append(logits)
predictions = np.concatenate(predictions, axis=0)
predictions
a= np.argmax(predictions, axis=1).flatten()
print(label_dict[a[0]])

conspiracy
